# Check GPU status

In [1]:
!nvidia-smi

Thu Jul 18 01:54:57 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Download official BERT code

In [2]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 333 (delta 0), reused 2 (delta 0), pack-reused 329
Receiving objects: 100% (333/333), 282.37 KiB | 520.00 KiB/s, done.
Resolving deltas: 100% (181/181), done.


# Download pretrained model

In [3]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip multi_cased_L-12_H-768_A-12.zip

--2019-07-18 01:55:06--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  58.7MB/s    in 7.0s    

2019-07-18 01:55:13 (90.3 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-

# Download KorQuAD dataset

In [4]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

--2019-07-18 01:55:28--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   131MB/s    in 0.3s    

2019-07-18 01:55:30 (131 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2019-07-18 01:55:32--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

# Reduce # of data to reduce training time

In [0]:
import json
train_data = json.load(open("KorQuAD_v1.0_train.json"))
small_train_data = dict()
small_train_data["version"] = train_data["version"]
small_train_data["data"] = train_data["data"][:int(len(train_data["data"])/5)]
json.dump(small_train_data, open("KorQuAD_v1.0_small_train.json", "w"))

# Fine-Tuning BERT model for KorQuAD data (around 50 min)

In [6]:
!python ./bert/run_squad.py \
--vocab_file=./multi_cased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=./multi_cased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=./multi_cased_L-12_H-768_A-12/bert_model.ckpt \
--do_train=True \
--train_file=./KorQuAD_v1.0_small_train.json \
--do_predict=True \
--predict_file=./KorQuAD_v1.0_dev.json \
--train_batch_size=8 \
--learning_rate=3e-5 \
--num_train_epochs=1.0 \
--max_seq_length=256 \
--doc_stride=128 \
--output_dir=./output \
--do_lower_case=False

W0718 01:55:39.437888 140617943459712 deprecation_wrapper.py:119] From /content/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 01:55:39.442039 140617943459712 deprecation_wrapper.py:119] From ./bert/run_squad.py:1283: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0718 01:55:39.442624 140617943459712 deprecation_wrapper.py:119] From ./bert/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0718 01:55:39.442779 140617943459712 deprecation_wrapper.py:119] From ./bert/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0718 01:55:39.442954 140617943459712 deprecation_wrapper.py:119] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0718 01:55:39.444112 140617943459712 deprecation_wrapper.py:11

# Download Evaluation code

In [9]:
!wget https://korquad.github.io/dataset/evaluate-v1.0.py

--2019-07-18 03:06:56--  https://korquad.github.io/dataset/evaluate-v1.0.py
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4305 (4.2K) [application/octet-stream]
Saving to: ‘evaluate-v1.0.py’

evaluate-v1.0.py    100%[===================>]   4.20K  --.-KB/s    in 0s      

2019-07-18 03:06:56 (73.0 MB/s) - ‘evaluate-v1.0.py’ saved [4305/4305]



# Evaluate the Fine-Tuned BERT model

In [10]:
!python ./evaluate-v1.0.py ./KorQuAD_v1.0_dev.json ./output/predictions.json

{"exact_match": 63.578108763422236, "f1": 84.69197023777235}


# Print Paragraph, Question, Answer and Model Prediction 

In [0]:
import json
import math

valid_data = json.load(open("./KorQuAD_v1.0_dev.json"))
prediction = json.load(open("./output/predictions.json"))

for document in valid_data['data']:
  for paragraph in document["paragraphs"]:
    print("Paragraph:")
    for line in range(math.ceil(len(paragraph["context"])/80)):
      print(paragraph['context'][line*80:(line+1)*80])
    print()
    
    for question in paragraph["qas"]:
      print("Question: %s" % question["question"])
      print("Answer: %s" % question["answers"])
      print("Prediction: %s" % prediction[question["id"]])
      print()
    input()
print(valid_data['data'][0]['paragraphs'][1])
for pid in prediction:
  print(prediction[pid])
  break

Paragraph:
1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰
청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰
은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가
스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 
18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집
행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.

Question: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
Answer: [{'text': '1989년 2월 15일', 'answer_start': 0}]
Prediction: 1989년 2월 15일

Question: 1989년 6월 30일 평양축전에 대표로 파견 된 인물은?
Answer: [{'text': '임수경', 'answer_start': 125}]
Prediction: 임수경

Question: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?
Answer: [{'text': '1989년', 'answer_start': 0}]
Prediction: 1989년 2월 15일

Question: 임종석을 검거한 장소는 경희대 내 어디인가?
Answer: [{'text': '학생회관 건물 계단', 'answer_start': 365}]
Prediction: 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관

Question: 임종석